In [5]:
# imports
import tkinter as tk
from tkinter import scrolledtext, messagebox, Spinbox, font
import numpy as np

Track data:
- Task completion rate
- Time-on-task
- Focus
- Energy Levels
- Satisfaction

In [8]:
satisfaction_scale = [
    "VERY SATISFIED",
    "SATISFIED",
    "NEITHER SATISFIED NOR DISSATISFIED",
    "DISSATISFIED",
    "VERY DISSATISFIED",
]
energy_scale = ["VERY LOW", "LOW", "MODERATE", "HIGH", "VERY HIGH"]
session_task_split = ["Single Task", "Multi-Task"]
task_completion = ["Main Task Completed", "Multi-Task Contribution"]

text_field_prompt = """
Please provide a brief explanation for your answers \n including any thoughts or observations you would like to share.
"""
# Condense:
label_ScaleWidgets = [
    "Satisfaction",
    "Energy Levels",
    "Focus",
    "Session Type",
    "Task Completion",
]
likert_widgets = [
    satisfaction_scale,
    energy_scale,
    energy_scale,
    session_task_split,
    task_completion,
]

In [9]:
grid_manager_likert = np.zeros((3, 2))
grid_manager_likert

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [10]:
%gui tk
global root
global text_input

def on_submit():
    print(text_input.get("1.0", "end-1c"))
    show_message()


def label_widget(label):
    """
    Creates a Label widget and adds it to the root window.

    Parameters:
    label (str): The text to display on the Label widget.

    Returns:
    None
    """
   
    label_widget = tk.Label(root, text=label, font=font.Font(size=16))
    label_widget.pack()


def option_menu_widget(likert_scale):
    """
    Creates an OptionMenu widget and adds it to the root window. The options are taken from the `likert_scale` list.

    Parameters:
    likert_scale (list of str): The options to display in the OptionMenu widget.

    Returns:
    None
    """
    default_value = tk.StringVar()
    default_value.set("Unanswered")
    option_menu = tk.OptionMenu(root, default_value, *likert_scale)
    option_menu.pack()
    option_menu.config(font=font.Font(size=16))
    

def likert_measures(scales, labels_for_widget):
    for widget in range(len(scales)):
        label_widget(labels_for_widget[widget])
        option_menu_widget(scales[widget])


def show_message():
    tk.messagebox.showinfo(icon='info',message="Submission Recorded")



Problem: Automate the palcement of widgets. 
title declare - ok.
likert measures(labels + widgets) - packed initially
 - they need to be in grid format first
options: manual vs algorithm. 
Algorithm approach 1:
The moving submatrix approach. 
1. slide an array with shape (2,1) e.g (2,) across a matrix size n*n
   - reason for submatrix (2,) is because one cell is the label and the bottom cell is the corresponding widget

In [11]:
%gui tk
root = tk.Tk()
# Body
root.title("Work Time Management")

# likert_measures(likert_widgets, label_ScaleWidgets)
label_widget(text_field_prompt)
text_input = scrolledtext.ScrolledText(root, wrap=tk.WORD)
text_input.pack(fill=tk.BOTH, expand=True)
submit_button = tk.Button(root, text="Submit", command=on_submit)
submit_button.pack(pady=10)
# Structure
root.mainloop()

: 

: 

Useful widgets:
scale = tk.Scale(root, orient='horizontal', label='Satisfcation')
scale.pack(pady=10)

def scale_values():
    return scale.get(), scale.cget('label')

> use the grid layout manager to place the widgets
option_menu1.grid(row=0, column=0)
option_menu2.grid(row=0, column=1)
text.grid(row=1, column=0, columnspan=2, sticky="WE")
scrollbar.grid(row=1, column=2, sticky="NS")